In [ ]:
# Base
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix


def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_prime(x):
    return sigmoid(x) * (1 - sigmoid(x))

def mse(y_true, y_pred):
    return np.mean(np.power(y_true - y_pred, 2))

def mse_prime(y_true, y_pred):
    return 2 * (y_pred - y_true) / y_true.size


In [ ]:
# Implementación de la capa densa (FCLayer)
class FCLayer:
    def __init__(self, input_size, output_size):
        self.weights = np.random.rand(input_size, output_size) - 0.5
        self.bias = np.random.rand(1, output_size) - 0.5

    def forward(self, input):
        self.input = input
        self.output = np.dot(self.input, self.weights) + self.bias
        return self.output

    def backward(self, output_gradient, learning_rate):
        input_gradient = np.dot(output_gradient, self.weights.T)
        weights_gradient = np.dot(self.input.T, output_gradient)


        self.weights -= learning_rate * weights_gradient
        self.bias -= learning_rate * np.mean(output_gradient, axis=0)
        return input_gradient


In [ ]:
# Implementación de la capa de activación
class ActivationLayer:
    def __init__(self, activation, activation_prime):
        self.activation = activation
        self.activation_prime = activation_prime

    def forward(self, input):
        self.input = input
        self.output = self.activation(self.input)
        return self.output

    def backward(self, output_gradient, learning_rate):
        return output_gradient * self.activation_prime(self.input)


In [ ]:
# Implementación de la capa Dropout
class DropoutLayer:
    def __init__(self, rate=0.5):
        self.rate = rate
        self.mask = None

    def forward(self, input):
        self.mask = np.random.binomial(1, 1 - self.rate, size=input.shape)
        return input * self.mask

    def backward(self, output_gradient, learning_rate):
        return output_gradient * self.mask


In [ ]:
# Implementación de la red neuronal con Early Stopping
class Network:
    def __init__(self):
        self.layers = []
        self.loss = None
        self.loss_prime = None

    def add(self, layer):
        self.layers.append(layer)

    def use(self, loss, loss_prime):
        self.loss = loss
        self.loss_prime = loss_prime

    def predict(self, input_data):
        samples = len(input_data)
        result = []

        for i in range(samples):
            output = input_data[i]
            for layer in self.layers:
                output = layer.forward(output)
            result.append(output)

        return result

    def fit(self, X_train, y_train, epochs, learning_rate, batch_size, X_val=None, y_val=None, patience=5):
        samples = len(X_train)
        best_loss = float('inf')
        patience_counter = 0

        for epoch in range(epochs):
            print(f'Epoch {epoch+1}/{epochs}')
            indices = np.arange(samples)
            np.random.shuffle(indices)
            X_train = X_train[indices]
            y_train = y_train[indices]

            for i in range(0, samples, batch_size):
                X_batch = X_train[i:i + batch_size]
                y_batch = y_train[i:i + batch_size]

                output = X_batch
                for layer in self.layers:
                    output = layer.forward(output)

                loss = self.loss(y_batch, output)
                output_gradient = self.loss_prime(y_batch, output)

                for layer in reversed(self.layers):
                    output_gradient = layer.backward(output_gradient, learning_rate)

            # Early Stopping
            if X_val is not None and y_val is not None:
                val_output = self.predict(X_val)
                val_loss = self.loss(y_val, val_output)

                if val_loss < best_loss:
                    best_loss = val_loss
                    patience_counter = 0
                else:
                    patience_counter += 1

                if patience_counter >= patience:
                    print("Early stopping triggered.")
                    return

            print(f'Loss: {loss}')


In [ ]:
#Uso del modelo con Dropout y Early Stopping

# Crear red neuronal
network = Network()
network.add(FCLayer(2, 3))  # Capa densa
network.add(ActivationLayer(sigmoid, sigmoid_prime))  # Capa de activación
network.add(DropoutLayer(0.2))  # Dropout con un rate del 20%
network.add(FCLayer(3, 1))  # Capa de salida

network.use(mse, mse_prime)

# Datos de ejemplo
X_train = np.random.rand(100, 2)
y_train = np.random.rand(100, 1)

X_val = np.random.rand(20, 2)
y_val = np.random.rand(20, 1)


network.fit(X_train, y_train, epochs=100, learning_rate=0.1, batch_size=32, X_val=X_val, y_val=y_val, patience=10)


Epoch 1/100
Loss: 0.15528773641449375
Epoch 2/100
Loss: 0.02912862288533439
Epoch 3/100
Loss: 0.03402584629396291
Epoch 4/100
Loss: 0.19151787720542057
Epoch 5/100
Loss: 0.10967333544584781
Epoch 6/100
Loss: 0.11557853704343969
Epoch 7/100
Loss: 0.10184223041129317
Epoch 8/100
Loss: 0.058497246014361876
Epoch 9/100
Loss: 0.0883089760291196
Epoch 10/100
Loss: 0.06736839239748905
Epoch 11/100
Loss: 0.10677688590636983
Epoch 12/100
Loss: 0.1384568452461558
Epoch 13/100
Loss: 0.16139330490274217
Epoch 14/100
Loss: 0.14107077920825595
Epoch 15/100
Loss: 0.08091716559258565
Epoch 16/100
Loss: 0.1532825450887873
Epoch 17/100
Loss: 0.08053105239723576
Epoch 18/100
Loss: 0.030082400274243835
Epoch 19/100
Early stopping triggered.
